In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import shap
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dropout, Flatten,
                                     Dense, LSTM, MultiHeadAttention, Concatenate, Reshape)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# =============================================================================
# --- Configuration ---
# =============================================================================

# --- SELECT YOUR CONFIGURATION HERE ---
DATASET = "MPOWER_DATASET"
MODE = "ALL_VALIDS"
FEATURE_MODE = "ALL"
MODEL_NAME = "cnn_att_lstm"
# ------------------------------------

# Path Setup
# dataset = "Italian" if DATASET == "ITALIAN_DATASET" else "Neurovoz"
dataset = "mPower"
FEATURES_FILE_PATH = os.path.join(os.getcwd(), dataset, "data", f"features_{MODE}_{FEATURE_MODE}.npz")
MODEL_PATH = os.path.join(os.getcwd(), dataset, f"results_{MODE}_{FEATURE_MODE}", MODEL_NAME)
os.makedirs(MODEL_PATH, exist_ok=True)

HISTORY_SAVE_PATH = os.path.join(MODEL_PATH, "history.csv")
BEST_MODEL_PATH = os.path.join(MODEL_PATH, "best_model.keras")
# --- NEW: Paths for explainability plots ---
SHAP_OUTPUT_PATH = os.path.join(MODEL_PATH, "shap_analysis")
GRADCAM_OUTPUT_PATH = os.path.join(MODEL_PATH, "gradcam_analysis")
os.makedirs(SHAP_OUTPUT_PATH, exist_ok=True)
os.makedirs(GRADCAM_OUTPUT_PATH, exist_ok=True)

# Hyperparameters
EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
L2_STRENGTH = 0.01

# Model Checkpoint Callback
checkpoint_cb = ModelCheckpoint(BEST_MODEL_PATH, monitor='val_auc', mode='max', save_best_only=True, verbose=1)

# =============================================================================
# --- Data Loading and Preparation ---
# =============================================================================

def load_data(path: str) -> tuple:
    """Loads features, demographics, and labels from the .npz file."""
    print(f"--- Loading data from {path} ---")
    with np.load(path) as data:
        mel_spectrograms = data['mel_spectrogram']
        mfccs = data['mfcc']
        labels = data['labels']
        try:
            ages = data['age']
            sexes = data['sex']
        except KeyError:
            ages = np.full_like(labels, np.nan)
            sexes = np.full_like(labels, np.nan)
        X = np.concatenate((mel_spectrograms, mfccs), axis=1)
    print("Data loaded successfully.")
    return X, labels, ages, sexes

# =============================================================================
# --- Model Architecture ---
# =============================================================================
from tensorflow.keras.saving import register_keras_serializable
# MODIFIED: Add the Keras serializable decorator
@register_keras_serializable()
class ParkinsonDetectorModel(Model):
    def __init__(self, input_shape, **kwargs):
        super(ParkinsonDetectorModel, self).__init__(**kwargs)
        self.input_shape_config = input_shape  # Store input_shape for saving

        # All your layer definitions remain the same
        self.reshape_in = Reshape((input_shape[0], input_shape[1], 1))
        self.conv1a = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same')
        self.conv1b = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same')
        self.pool1 = MaxPooling2D(5)
        self.drop1 = Dropout(DROPOUT_RATE)
        self.conv2a = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same')
        self.conv2b = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same', name='last_conv_layer')
        self.pool2 = MaxPooling2D(5, name='cnn_output')
        self.drop2 = Dropout(DROPOUT_RATE)
        self.flatten_cnn = Flatten()
        self.attention = MultiHeadAttention(num_heads=2, key_dim=64, name='attention_output')
        self.flatten_att = Flatten()
        self.lstm1 = LSTM(128, return_sequences=True)
        self.lstm2 = LSTM(128, return_sequences=False, name='lstm_output')
        self.drop_lstm = Dropout(DROPOUT_RATE)
        self.concat = Concatenate()
        self.dense_bottleneck = Dense(128, activation='relu', name='bottleneck_features')
        self.dense_output = Dense(1, activation='sigmoid')

    def call(self, inputs):
        # The 'call' method is unchanged
        x = self.reshape_in(inputs)
        x = self.conv1a(x)
        x = self.conv1b(x)
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.conv2a(x)
        x = self.conv2b(x)
        x = self.pool2(x)
        x = self.drop2(x)
        cnn_flat = self.flatten_cnn(x)
        shape = tf.shape(x)
        sequence = tf.reshape(x, [-1, shape[1] * shape[2], shape[3]])
        att_out = self.attention(query=sequence, key=sequence, value=sequence)
        att_flat = self.flatten_att(att_out)
        lstm_seq = self.lstm1(sequence)
        lstm_out = self.lstm2(lstm_seq)
        lstm_out = self.drop_lstm(lstm_out)
        concatenated = self.concat([cnn_flat, att_flat, lstm_out])
        bottleneck = self.dense_bottleneck(concatenated)
        return self.dense_output(bottleneck)

    # NEW: Add a get_config method for serialization
    def get_config(self):
        # Get the base configuration from the parent class
        config = super(ParkinsonDetectorModel, self).get_config()
        # Add the input_shape to the config so Keras can rebuild the model
        config.update({"input_shape": self.input_shape_config})
        return config

    # This allows Keras to load the model from the config
    @classmethod
    def from_config(cls, config):
        return cls(**config)

def build_model(input_shape: tuple) -> Model:
    """Builds the hybrid model by wrapping the custom class in a Functional API model."""
    print("--- Building the model ---")

    # Define the input layer
    inputs = Input(shape=input_shape)

    # Instantiate your custom model class as if it were a layer
    parkinson_detector = ParkinsonDetectorModel(input_shape=input_shape)

    # Pass the inputs through your custom model
    outputs = parkinson_detector(inputs)

    # Create the final, standard Keras Model
    model = Model(inputs=inputs, outputs=outputs)

    print("Model built successfully.")
    return model

# =============================================================================
# --- Model Explainability (SHAP & Grad-CAM) ---
# =============================================================================
from tqdm import tqdm # Make sure tqdm is imported at the top of your script
import shap
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import shap
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from scipy.stats import ttest_ind
from tqdm import tqdm # Make sure tqdm is imported at the top of your script
import shap
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from scipy.stats import ttest_ind

def run_full_shap_analysis(model, X_train, X_test, y_test, output_path, num_samples=88, top_n=20):
    """
    Run SHAP explainability analysis on spectrogram-based CNN.
    Generates global + class-wise plots without exploding memory.
    """

    print("\n--- Running Full SHAP Analysis ---")

    # Ensure output dir exists
    os.makedirs(output_path, exist_ok=True)

    # Pick random subset of test samples for SHAP
    if num_samples < len(X_test):
        idx = np.random.choice(len(X_test), num_samples, replace=False)
        test_samples = X_test[idx]
        y_true_samples = y_test[idx]
    else:
        test_samples = X_test
        y_true_samples = y_test

    print(f"Calculating SHAP values for {len(test_samples)} samples...")

    # SHAP explainer
    explainer = shap.GradientExplainer(model, X_train[:50])  # small background set

    shap_values_list = []
    for sample in tqdm(test_samples, desc="SHAP Progress"):
        sample_batch = np.expand_dims(sample, axis=0).astype(np.float32)
        sv = explainer.shap_values(sample_batch)

        # --- handle different return formats ---
        if isinstance(sv, list):  # for binary classifiers shap gives [array]
            sv = sv[0]
        # sv should now be shape (1, 60, 94, 1)

        shap_values_list.append(sv)

    shap_values = np.vstack(shap_values_list)  # (num_samples, 60, 94, 1)

    print(f"\nSHAP values shape: {shap_values.shape}, Test samples shape: {test_samples.shape}")

       # === 1. Global Top-N Pixel Bar Plot (with time/freq labels) ===
    flat_shap = shap_values.reshape(shap_values.shape[0], -1)  # (N, 60*94)
    mean_abs = np.mean(np.abs(flat_shap), axis=0)

    top_idx = np.argsort(mean_abs)[::-1][:top_n]

    # Map back to (time, freq)
    coords = [np.unravel_index(i, (shap_values.shape[1], shap_values.shape[2])) for i in top_idx]
    labels = [f"T{t} F{f}" for t, f in coords]  # Example: "T12 F45"

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(top_idx)), mean_abs[top_idx])
    plt.xticks(range(len(top_idx)), labels, rotation=45, ha="right")
    plt.title(f"Top-{top_n} Global SHAP Features (time × frequency bins)")
    plt.xlabel("Time Bin × Frequency Bin")
    plt.ylabel("Mean |SHAP value|")
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, "shap_global_bar.png"), dpi=300, bbox_inches="tight")
    plt.close()
    print("-> Saved 'shap_global_bar.png'")


    # === 2. Class-wise Average Heatmaps ===
    hc_mask = y_true_samples == 0
    pd_mask = y_true_samples == 1

    if np.any(hc_mask):
        hc_mean = shap_values[hc_mask].mean(axis=0).squeeze()
        plt.imshow(hc_mean, cmap="bwr", aspect="auto")
        plt.colorbar(label="Mean SHAP Value")
        plt.title("Average SHAP Heatmap - Healthy")
        plt.savefig(os.path.join(output_path, "shap_summary_healthy.png"), dpi=300, bbox_inches="tight")
        plt.close()
        print("-> Saved 'shap_summary_healthy.png'")

    if np.any(pd_mask):
        pd_mean = shap_values[pd_mask].mean(axis=0).squeeze()
        plt.imshow(pd_mean, cmap="bwr", aspect="auto")
        plt.colorbar(label="Mean SHAP Value")
        plt.title("Average SHAP Heatmap - Parkinson")
        plt.savefig(os.path.join(output_path, "shap_summary_parkinson.png"), dpi=300, bbox_inches="tight")
        plt.close()
        print("-> Saved 'shap_summary_parkinson.png'")

    # === 3. Difference Heatmap (PD - HC) ===
    if np.any(hc_mask) and np.any(pd_mask):
        diff_map = pd_mean - hc_mean
        plt.imshow(diff_map, cmap="seismic", aspect="auto")
        plt.colorbar(label="Δ SHAP (PD - HC)")
        plt.title("SHAP Difference Heatmap (Parkinson - Healthy)")
        plt.savefig(os.path.join(output_path, "shap_difference.png"), dpi=300, bbox_inches="tight")
        plt.close()
        print("-> Saved 'shap_difference.png'")

    # === 4. Significance Map (t-test per pixel) ===
    if np.any(hc_mask) and np.any(pd_mask):
        hc_vals = shap_values[hc_mask].reshape(np.sum(hc_mask), -1)
        pd_vals = shap_values[pd_mask].reshape(np.sum(pd_mask), -1)

        t_stat, p_vals = ttest_ind(pd_vals, hc_vals, axis=0, equal_var=False)
        p_map = p_vals.reshape(shap_values.shape[1], shap_values.shape[2])

        plt.imshow(p_map, cmap="viridis_r", aspect="auto", vmin=0, vmax=0.05)
        plt.colorbar(label="p-value (t-test)")
        plt.title("Statistical Significance Map (PD vs HC)")
        plt.savefig(os.path.join(output_path, "shap_significance.png"), dpi=300, bbox_inches="tight")
        plt.close()
        print("-> Saved 'shap_significance.png'")

    print("\n--- SHAP Analysis Complete ---")


def make_gradcam_heatmap(img_array: np.ndarray, model: Model, last_conv_layer_name: str):
    """Generates the Grad-CAM heatmap for a single input image."""
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [2]:
def debug_model_structure(model):
    """Debug function to examine model structure"""
    print("\n=== MODEL STRUCTURE DEBUG ===")
    print(f"Model type: {type(model)}")
    print(f"Number of layers: {len(model.layers)}")

    for i, layer in enumerate(model.layers):
        print(f"Layer {i}: {layer.name} - Type: {type(layer)}")

        # Check if this layer has sublayers
        if hasattr(layer, 'layers'):
            print(f"  └── Has {len(layer.layers)} sublayers")
            for j, sublayer in enumerate(layer.layers):
                print(f"      Layer {j}: {sublayer.name} - Type: {type(sublayer)}")

    print("=== END DEBUG ===\n")


In [3]:

# =============================================================================
# --- Main Execution ---
# =============================================================================
if __name__ == '__main__':
    X, y, age, sex = load_data(FEATURES_FILE_PATH)
    X_train, X_test, y_train, y_test, age_train, age_test, sex_train, sex_test = train_test_split(
        X, y, age, sex, test_size=0.2, random_state=42, stratify=y
    )
    print(f"\nData split into training ({len(y_train)}) and testing ({len(y_test)}) sets.")

    model = build_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    debug_model_structure(model)
    model.summary()
    optimizer = Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

    print("\n--- Starting model training ---")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint_cb],
        verbose=1
    )
    print("--- Model training finished ---")

    pd.DataFrame(history.history).to_csv(HISTORY_SAVE_PATH, index_label='epoch')
    print(f"\nTraining history saved to '{HISTORY_SAVE_PATH}'")


--- Loading data from D:\Projects\Voice\Parkinson-s-Disease-Detector-Using-AI\Parkinson-s-Disease-Detector-Using-AI\1\mPower\data\features_ALL_VALIDS_ALL.npz ---
Data loaded successfully.

Data split into training (1664) and testing (416) sets.
--- Building the model ---

Model built successfully.

=== MODEL STRUCTURE DEBUG ===
Model type: <class 'keras.src.models.functional.Functional'>
Number of layers: 2
Layer 0: input_layer - Type: <class 'keras.src.layers.core.input_layer.InputLayer'>
Layer 1: parkinson_detector_model - Type: <class '__main__.ParkinsonDetectorModel'>
  └── Has 18 sublayers
      Layer 0: reshape - Type: <class 'keras.src.layers.reshaping.reshape.Reshape'>
      Layer 1: conv2d - Type: <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
      Layer 2: conv2d_1 - Type: <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
      Layer 3: max_pooling2d - Type: <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
      Layer 4: dropout - Type: <class 'kera

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 60, 94)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ parkinson_detector_model        │ (None, 1)              │       687,617 │
│ (ParkinsonDetectorModel)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 687,617 (2.62 MB)

 Trainable params: 687,617 (2.62 MB)

 Non-trainable params: 0 (0.00 B)


--- Starting model training ---
Epoch 1/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 743ms/step - accuracy: 0.4772 - auc: 0.4989 - loss: 3.5938
Epoch 1: val_auc improved from None to 0.52345, saving model to D:\Projects\Voice\Parkinson-s-Disease-Detector-Using-AI\Parkinson-s-Disease-Detector-Using-AI\1\mPower\results_ALL_VALIDS_ALL\cnn_att_lstm\best_model.keras
52/52 ━━━━━━━━━━━━━━━━━━━━ 58s 830ms/step - accuracy: 0.4790 - auc: 0.4840 - loss: 2.9495 - val_accuracy: 0.5072 - val_auc: 0.5234 - val_loss: 2.4588
Epoch 2/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 773ms/step - accuracy: 0.5222 - auc: 0.5235 - loss: 2.3994
Epoch 2: val_auc improved from 0.52345 to 0.56829, saving model to D:\Projects\Voice\Parkinson-s-Disease-Detector-Using-AI\Parkinson-s-Disease-Detector-Using-AI\1\mPower\results_ALL_VALIDS_ALL\cnn_att_lstm\best_model.keras
52/52 ━━━━━━━━━━━━━━━━━━━━ 43s 828ms/step - accuracy: 0.5168 - auc: 0.5272 - loss: 2.3140 - val_accuracy: 0.5168 - val_auc: 0.5683 - val_loss: 2.1372
Epoch 3/30
52/52 ━━━━━━━

KeyboardInterrupt: 

In [4]:
def run_gradcam_analysis(model, X_test, y_test, output_path, num_samples=10):
    """
    Run Grad-CAM analysis on the last conv layer of the model.
    Saves Grad-CAM overlays for correctly classified samples.
    """
    print("\n--- Running Grad-CAM Analysis ---")

    os.makedirs(output_path, exist_ok=True)

    # 🔹 Ensure model is built by running a forward pass once
    _ = model(X_test[:1], training=False)

    # 🔹 Find the ParkinsonDetectorModel instance within the functional model
    parkinson_detector = None
    for layer in model.layers:
        if isinstance(layer, ParkinsonDetectorModel):
            parkinson_detector = layer
            break

    if parkinson_detector is None:
        print("❌ ParkinsonDetectorModel not found in the model layers.")
        return

    # 🔹 Get the last conv layer from the custom model
    # We know from your model definition that 'conv2b' is the last conv layer
    last_conv_layer = parkinson_detector.conv2b
    print(f"✅ Using last conv layer: {last_conv_layer.name}")

    # 🔹 Create a model that outputs both the conv layer activations and final predictions
    # We need to create a new model that goes from input to the conv layer output
    def get_conv_and_output(inputs):
        # Pass through the ParkinsonDetectorModel and extract intermediate outputs
        x = parkinson_detector.reshape_in(inputs)
        x = parkinson_detector.conv1a(x)
        x = parkinson_detector.conv1b(x)
        x = parkinson_detector.pool1(x)
        x = parkinson_detector.drop1(x, training=False)  # Set training=False for consistent behavior
        x = parkinson_detector.conv2a(x)
        conv_output = parkinson_detector.conv2b(x)  # This is what we want for Grad-CAM

        # Continue with the rest of the model
        x = parkinson_detector.pool2(conv_output)
        x = parkinson_detector.drop2(x, training=False)
        cnn_flat = parkinson_detector.flatten_cnn(x)

        # Attention and LSTM branches
        shape = tf.shape(x)
        sequence = tf.reshape(x, [-1, shape[1] * shape[2], shape[3]])
        att_out = parkinson_detector.attention(query=sequence, key=sequence, value=sequence)
        att_flat = parkinson_detector.flatten_att(att_out)
        lstm_seq = parkinson_detector.lstm1(sequence)
        lstm_out = parkinson_detector.lstm2(lstm_seq)
        lstm_out = parkinson_detector.drop_lstm(lstm_out, training=False)

        # Final layers
        concatenated = parkinson_detector.concat([cnn_flat, att_flat, lstm_out])
        bottleneck = parkinson_detector.dense_bottleneck(concatenated)
        final_output = parkinson_detector.dense_output(bottleneck)

        return conv_output, final_output

    # Predictions to select TP/TN
    y_pred_probs = model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    tp_idx = np.where((y_test == 1) & (y_pred == 1))[0]
    tn_idx = np.where((y_test == 0) & (y_pred == 0))[0]

    selected_idx_tp = []
    if len(tp_idx) > 0:
        selected_idx_tp.extend(np.random.choice(tp_idx, min(30, len(tp_idx)), replace=False))
    # if len(tn_idx) > 0:
    #     selected_idx.extend(np.random.choice(tn_idx, min(30, len(tn_idx)), replace=False))

    print(f"Selected {len(selected_idx_tp)} samples for Grad-CAM. TRUE POSITIVE")

    tp_heatmaps = []
    # 🔹 Loop through selected samples
    for i in selected_idx_tp:
        img = X_test[i:i+1]
        label = y_test[i]
        pred_prob = y_pred_probs[i][0]

        with tf.GradientTape() as tape:
            img_tensor = tf.cast(img, tf.float32)
            tape.watch(img_tensor)
            conv_outputs, preds = get_conv_and_output(img_tensor)
            loss = preds[:, 0]

        # Get gradients of the loss with respect to the conv layer output
        grads = tape.gradient(loss, conv_outputs)

        # Global average pooling of gradients
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        conv_outputs = conv_outputs[0].numpy()
        pooled_grads = pooled_grads.numpy()

        # Weighted combination of feature maps
        heatmap = np.zeros(conv_outputs.shape[:-1])
        for j in range(conv_outputs.shape[-1]):
            heatmap += pooled_grads[j] * conv_outputs[:, :, j]

        # ReLU and normalize
        heatmap = np.maximum(heatmap, 0)
        heatmap /= (heatmap.max() + 1e-10)

        # Resize to match input size
        heatmap_resized = tf.image.resize(
            heatmap[..., np.newaxis],
            (img.shape[1], img.shape[2])
        ).numpy().squeeze()
        tp_heatmaps.append(heatmap)

    selected_idx_tn = []
    if len(tn_idx) > 0:
        selected_idx_tn.extend(np.random.choice(tn_idx, min(30, len(tn_idx)), replace=False))

    print(f"Selected {len(selected_idx_tn)} samples for Grad-CAM. TRUE NEGATIVE")

    tn_heatmaps = []
    # 🔹 Loop through selected samples
    for i in selected_idx_tn:
        img = X_test[i:i+1]
        label = y_test[i]
        pred_prob = y_pred_probs[i][0]

        with tf.GradientTape() as tape:
            img_tensor = tf.cast(img, tf.float32)
            tape.watch(img_tensor)
            conv_outputs, preds = get_conv_and_output(img_tensor)
            loss = preds[:, 0]

        # Get gradients of the loss with respect to the conv layer output
        grads = tape.gradient(loss, conv_outputs)

        # Global average pooling of gradients
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        conv_outputs = conv_outputs[0].numpy()
        pooled_grads = pooled_grads.numpy()

        # Weighted combination of feature maps
        heatmap = np.zeros(conv_outputs.shape[:-1])
        for j in range(conv_outputs.shape[-1]):
            heatmap += pooled_grads[j] * conv_outputs[:, :, j]

        # ReLU and normalize
        heatmap = np.maximum(heatmap, 0)
        heatmap /= (heatmap.max() + 1e-10)

        # Resize to match input size
        heatmap_resized = tf.image.resize(
            heatmap[..., np.newaxis],
            (img.shape[1], img.shape[2])
        ).numpy().squeeze()
        tn_heatmaps.append(heatmap)

    avg_tp_heatmap = np.mean(tp_heatmaps, axis=0) if tp_heatmaps else np.zeros((X_test.shape[1], X_test.shape[2]))
    avg_tn_heatmap = np.mean(tn_heatmaps, axis=0) if tn_heatmaps else np.zeros((X_test.shape[1], X_test.shape[2]))

    # 🔹 Visualize the average heatmaps side-by-side
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Plot average heatmap for True Positives (Parkinson's)
    im1 = axes[0].imshow(avg_tp_heatmap, cmap='jet', aspect='auto')
    axes[0].set_title(f'Average Grad-CAM for Parkinson\'s (TP)\n({len(tp_heatmaps)} samples)', fontsize=14)
    axes[0].set_xlabel("Time")
    axes[0].set_ylabel("Frequency")
    fig.colorbar(im1, ax=axes[0])

    # Plot average heatmap for True Negatives (Healthy)
    im2 = axes[1].imshow(avg_tn_heatmap, cmap='jet', aspect='auto')
    axes[1].set_title(f'Average Grad-CAM for Healthy (TN)\n({len(tn_heatmaps)} samples)', fontsize=14)
    axes[1].set_xlabel("Time")
    axes[1].set_ylabel("Frequency")
    fig.colorbar(im2, ax=axes[1])

    plt.suptitle("Average Model Attention by Class", fontsize=18, fontweight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make room for suptitle

    save_path = os.path.join(output_path, "gradcam_average_comparison.png")
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"✅ Saved average Grad-CAM comparison to: {save_path}\n")
        # # Plot overlay
        # plt.figure(figsize=(12, 4))
        #
        # # Original spectrogram
        # plt.subplot(1, 3, 1)
        # plt.imshow(img[0].squeeze(), cmap='viridis', aspect='auto')
        # plt.title(f'Original\nLabel={label}')
        # plt.colorbar()
        #
        # # Grad-CAM heatmap
        # plt.subplot(1, 3, 2)
        # plt.imshow(heatmap_resized, cmap='jet', aspect='auto')
        # plt.title(f'Grad-CAM\nPred={pred_prob:.3f}')
        # plt.colorbar()
        #
        # # Overlay
        # plt.subplot(1, 3, 3)
        # plt.imshow(img[0].squeeze(), cmap='viridis', aspect='auto', alpha=0.7)
        # plt.imshow(heatmap_resized, cmap='jet', alpha=0.3, aspect='auto')
        # plt.title(f'Overlay\n(Label={label}, Pred={pred_prob:.3f})')
        # plt.colorbar()
        #
        # plt.tight_layout()
        # save_path = os.path.join(output_path, f"gradcam_sample_{i}.png")
        # plt.savefig(save_path, dpi=300, bbox_inches="tight")
        # plt.close()

    print("✅ Grad-CAM analysis completed.\n")


In [5]:
    if os.path.exists(BEST_MODEL_PATH):
        print("\n--- Loading best saved model for explainability analysis ---")
        best_model = load_model(BEST_MODEL_PATH, custom_objects={'ParkinsonDetectorModel': ParkinsonDetectorModel})

        # --- MODIFIED: Call the single, unified SHAP function ---
        run_full_shap_analysis(best_model, X_train, X_test, y_test, SHAP_OUTPUT_PATH)



--- Loading best saved model for explainability analysis ---

--- Running Full SHAP Analysis ---
Calculating SHAP values for 88 samples...


SHAP Progress:   0%|          | 0/88 [00:00<?, ?it/s]C:\Users\bahar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=['Tensor(shape=(1, 60, 94))']
  warnings.warn(msg)
C:\Users\bahar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=['Tensor(shape=(50, 60, 94))']
  warnings.warn(msg)
SHAP Progress: 100%|██████████| 88/88 [04:00<00:00,  2.73s/it]



SHAP values shape: (88, 60, 94, 1), Test samples shape: (88, 60, 94)
-> Saved 'shap_global_bar.png'
-> Saved 'shap_summary_healthy.png'
-> Saved 'shap_summary_parkinson.png'
-> Saved 'shap_difference.png'
-> Saved 'shap_significance.png'

--- SHAP Analysis Complete ---


In [6]:
        run_gradcam_analysis(best_model, X_test, y_test, GRADCAM_OUTPUT_PATH)


--- Running Grad-CAM Analysis ---
✅ Using last conv layer: last_conv_layer
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step
Selected 30 samples for Grad-CAM. TRUE POSITIVE
Selected 30 samples for Grad-CAM. TRUE NEGATIVE
✅ Saved average Grad-CAM comparison to: D:\Projects\Voice\Parkinson-s-Disease-Detector-Using-AI\Parkinson-s-Disease-Detector-Using-AI\1\Neurovoz\results_A_ALL\cnn_att_lstm\gradcam_analysis\gradcam_average_comparison.png

✅ Grad-CAM analysis completed.



In [ ]:
    else:
            print("\nCould not find best model file. Skipping SHAP and Grad-CAM analysis.")